In [47]:
from bs4 import BeautifulSoup
import numpy as np
import requests

In [3]:
request = requests.get("https://www.biznesradar.pl/wskazniki-wartosci-rynkowej/GIG", allow_redirects=False)
html_code = BeautifulSoup(request.content, 'html.parser')


In [ ]:
class CompanyIndexScraper:
    SKELETON_URL = "https://www.biznesradar.pl"
    INDEXES_TO_SCRAP = {
        "wskazniki-wartosci-rynkowej" : [
            "Cena / Wartość księgowa",
            "Cena / Wartość księgowa Grahama",
            "Cena / Przychody ze sprzedaży",
            "Cena / Zysk",
            "Cena / Zysk operacyjny",
            "EV / Przychody ze sprzedaży",
            "EV / EBIT",
            "EV / EBITDA",
        ],
        "wskazniki-rentownosci" : [
            "ROE",
            "ROA",
            "Marża zysku operacyjnego",
            "Marża zysku netto",
            "Marża zysku ze sprzedaży",
            "Marża zysku brutto",
            "Marża zysku brutto ze sprzedaży",
            "Rentowność operacyjna aktywów",
        ],
        "wskazniki-przeplywow-pienieznych" : [
            "Udział zysku netto w przepływach operacyjnych",
            "Wskaźnik źródeł finansowania inwestycji",
        ],
        "wskazniki-zadluzenia" : [
            "Zadłużenie ogólne",
            "Zadłużenie kapitału własnego",
            "Zastosowanie kapitału obcego",
            "Wskaźnik ogólnej sytuacji finansowej",
        ],
        "wskazniki-plynnosci" : [
            "I stopień pokrycia",
            "Płynność gotówkowa",
            "Płynność szybka",
            "Płynność bieżąca",
            "Pokrycie zobowiązań należnościami",
        ],
    }

    DATA_TYPE = [
        "Company",
        "Sector",
        "CompanyY/Y",
        "SectorY/Y",
        "CompanyQ/Q",
        "SectorQ/Q",
    ]

    def __init__(self):
        self.dates_header_ = None
        self.html_table_   = None
        self.data_array_   = None
        self.columns_name_ = sum(CompanyIndexScraper.INDEXES_TO_SCRAP.values(), [])

    def __generate_url_address(indexes_type : str, ticker : str) -> str:
        return f"{CompanyIndexScraper.SKELETON_URL}/{indexes_type}/{ticker}"
    
    def __download_http_response(self, url : str) -> BeautifulSoup:
        request = requests.get(url, allow_redirects=False)
        if request.status_code != 200:
            raise f"Download http response error code: {request.status_code}"
        return BeautifulSoup(request.content, 'html.parser')

    def __download_and_prepare_html_table(self, indexes_type : str, ticker : str) -> None:
        url = self.__generate_url_address(indexes_type, ticker)
        html_response = self.__download_http_response(url)
        self.html_table_ = html_response.find("table", attrs={"class":"report-table"}) 

    def __scrap_header_date(self):
        pass

    def __prepare_data_space(self) -> None:
        # Before use dates_header_ should be initialized 
        self.data_array_ = np.empty((len(self.dates_header_), len(self.columns_name_), len(CompanyIndexScraper.DATA_TYPE)))
        self.data_array_[:] = np.nan



In [23]:
indexes_to_scrap = {
    "wskazniki-wartosci-rynkowej" : [
        "Cena / Wartość księgowa",
        "Cena / Wartość księgowa Grahama",
        "Cena / Przychody ze sprzedaży",
        "Cena / Zysk",
        "Cena / Zysk operacyjny",
        "EV / Przychody ze sprzedaży",
        "EV / EBIT",
        "EV / EBITDA",
    ],
    "wskazniki-rentownosci" : [
        "ROE",
        "ROA",
        "Marża zysku operacyjnego",
        "Marża zysku netto",
        "Marża zysku ze sprzedaży",
        "Marża zysku brutto",
        "Marża zysku brutto ze sprzedaży",
        "Rentowność operacyjna aktywów",
    ],
    "wskazniki-przeplywow-pienieznych" : [
        "Udział zysku netto w przepływach operacyjnych",
        "Wskaźnik źródeł finansowania inwestycji",
    ],
    "wskazniki-zadluzenia" : [
        "Zadłużenie ogólne",
        "Zadłużenie kapitału własnego",
        "Zastosowanie kapitału obcego",
        "Wskaźnik ogólnej sytuacji finansowej",
    ],
    "wskazniki-plynnosci" : [
        "I stopień pokrycia",
        "Płynność gotówkowa",
        "Płynność szybka",
        "Płynność bieżąca",
        "Pokrycie zobowiązań należnościami",
    ],
}

In [48]:
columns = sum(indexes_to_scrap.values(), [])

In [49]:
header_row = table.find("tr") 
dates_header = {"Date": ["".join(date.text.split()).split('(')[0] for date in date_tr.find_all("th", attrs={"class":"thq h"})]}

In [54]:
x = np.zeros((len(dates_header["Date"]), len(columns), 6))

In [66]:
table_rows = table.find_all("tr")[1:] # Omit the header dates row
table_row = table_rows[0]
table_row.find("td", attrs={"class":"f"}).text in indexes_to_scrap["wskazniki-wartosci-rynkowej"]


False

In [22]:
td = row.find_all("td", attrs={"class":"f"})
td.find("strong")

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [17]:
row.find_all("td", attrs={"class":"h"})

[<td class="h"><span class="value"><span class="pv"><span>14.49</span></span></span></td>,
 <td class="h"><span class="value"><span class="pv"><span>11.14</span></span></span><div class="changeyy">r/r <span class="pv"><span><span class="q_ch_per cminus">-23.16%</span></span></span></div></td>,
 <td class="h"><span class="value"><span class="pv"><span>11.65</span></span></span><div class="changeyy">r/r <span class="pv"><span><span class="q_ch_per cplus">+4.64%</span></span></span></div></td>,
 <td class="h"><span class="value"><span class="pv"><span>14.43</span></span></span><div class="changeyy">r/r <span class="pv"><span><span class="q_ch_per cplus">+23.82%</span></span></span></div></td>,
 <td class="h"><span class="value"><span class="pv"><span>14.32</span></span></span></td>,
 <td class="h"><span class="value"><span class="pv"><span>14.91</span></span></span><div class="changeqq">k/k <span class="pv"><span><span class="q_ch_per cplus">+4.11%</span></span></span></div></td>,
 <td cl